In [10]:
!pip install tensorflow

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorboard-2.16.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached keras-3.3.3-py3-none-any.whl.metadata (5.7 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ---------------------------------------- 0.0/46.2 kB ? eta -:--:--
     ---------------------------------------- 46.2/46.2 kB 2.2 MB/s eta 0:00:00
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------

In [25]:
import librosa
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import tqdm as tqdm

In [26]:
import pandas as pd
import numpy as np

esc50 = "esc50.csv" 
audio = "audio/audio/44100/" 
df = pd.read_csv(esc50)
df.head()
print("shape of df: ", df.shape)
df.head()

shape of df:  (2000, 7)


,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [27]:
category = df['category'].unique()
categories = {i:x for x,i in enumerate(category)}
df['target'] = df['category'].map(categories)
df=df.drop(['fold','esc10','src_file','take'], axis=1)
df.shape

(2000, 3)

In [28]:
categories

{'dog': 0,
 'chirping_birds': 1,
 'vacuum_cleaner': 2,
 'thunderstorm': 3,
 'door_wood_knock': 4,
 'can_opening': 5,
 'crow': 6,
 'clapping': 7,
 'fireworks': 8,
 'chainsaw': 9,
 'airplane': 10,
 'mouse_click': 11,
 'pouring_water': 12,
 'train': 13,
 'sheep': 14,
 'water_drops': 15,
 'church_bells': 16,
 'clock_alarm': 17,
 'keyboard_typing': 18,
 'wind': 19,
 'footsteps': 20,
 'frog': 21,
 'cow': 22,
 'brushing_teeth': 23,
 'car_horn': 24,
 'crackling_fire': 25,
 'helicopter': 26,
 'drinking_sipping': 27,
 'rain': 28,
 'insects': 29,
 'laughing': 30,
 'hen': 31,
 'engine': 32,
 'breathing': 33,
 'crying_baby': 34,
 'hand_saw': 35,
 'coughing': 36,
 'glass_breaking': 37,
 'snoring': 38,
 'toilet_flush': 39,
 'pig': 40,
 'washing_machine': 41,
 'clock_tick': 42,
 'sneezing': 43,
 'rooster': 44,
 'sea_waves': 45,
 'siren': 46,
 'cat': 47,
 'door_wood_creaks': 48,
 'crickets': 49}

In [29]:

X = []
y = []
for index, row in df.iterrows():
    sig, sr = librosa.load(audio + row[0])
    mfcc_ = librosa.feature.mfcc(y=sig, sr=sr, n_mfcc=40)
    X.append(mfcc_)
    y.append(row[1])
    
X = np.array(X) 
y = np.array(y)
X.shape

C:\Users\anura\AppData\Local\Temp\ipykernel_28616\125016286.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sig, sr = librosa.load(audio + row[0])
C:\Users\anura\AppData\Local\Temp\ipykernel_28616\125016286.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y.append(row[1])


(2000, 40, 216)

In [30]:
y = tf.keras.utils.to_categorical(y , num_classes=50)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
y.shape
inp = (40,216,1)

In [36]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X , y ,test_size=0.2, random_state=42)
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
model = models.Sequential([
    # First Convolutional Block
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=inp),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),

    # Second Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),

    # Third Convolutional Block
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),

    # Global Average Pooling and Fully Connected Layers
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(50, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_18 (Conv2D)              │ (None, 40, 216, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 40, 216, 32)    │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 20, 108, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 20, 108, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 20, 108, 64)    │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 10, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 10, 54, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 10, 54, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 5, 27, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 50)             │        12,850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 332,306 (1.27 MB)

 Trainable params: 332,306 (1.27 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
epoch = 10

history = model.fit(X_train,y_train ,
            validation_data=(X_test,y_test),
            epochs=epoch)

Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 20s 358ms/step - accuracy: 0.0215 - loss: 4.0659 - val_accuracy: 0.0275 - val_loss: 3.8633
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 328ms/step - accuracy: 0.0358 - loss: 3.8677 - val_accuracy: 0.0325 - val_loss: 3.8047
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 324ms/step - accuracy: 0.0382 - loss: 3.8067 - val_accuracy: 0.0500 - val_loss: 3.7355
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 325ms/step - accuracy: 0.0470 - loss: 3.7727 - val_accuracy: 0.0500 - val_loss: 3.6343
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 322ms/step - accuracy: 0.0531 - loss: 3.6763 - val_accuracy: 0.0925 - val_loss: 3.5972
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 320ms/step - accuracy: 0.0581 - loss: 3.6340 - val_accuracy: 0.0975 - val_loss: 3.4921
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 325ms/step - accuracy: 0.0748 - loss: 3.5275 - val_accuracy: 0.0925 - val_loss: 3.4130
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 16s 328ms/step - accuracy: 0.0981 - loss: 3.4633 - val_accu

In [38]:
test_accuracy = model.evaluate(X_test,y_test,verbose =0)
print(test_accuracy[1])

0.13500000536441803
